In [187]:
from dotenv import load_dotenv
import lyricsgenius
import os
import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

To get the lyrics of the songs we are going to use the `lyricgenius` library. This library requires a Genius API key which can be generated creating a free account in [Genius](https://genius.com).

In [2]:
# load enviroment variables from .env file
load_dotenv()

# acces the variable
token = os.getenv("GENIUS_API_TOKEN")

Now we create the genius object that we will use to fetch the songs.

In [3]:
genius = lyricsgenius.Genius(token)
genius.remove_section_headers = True 
genius.verbose = False

To find the songs of a given artist we need its `id`:

In [4]:
artist_names = ["Kase.O", "SFDK", "Ayax y Prok", "Kaze"]

artists = {}

for name in artist_names:
    artist_id = genius.search_artist(name, max_songs=0).id
    artists[artist_id] = name

print(artists)

{27132: 'Kase.O', 55782: 'SFDK', 1556128: 'Ayax y Prok', 1062440: 'Kaze'}


Now we can go through the pages of the website to find songs of our artist. We will omit colaborations, live songs and remixes.

In [5]:
def find_songs(artist_id, page):  
    songs = genius.artist_songs(artist_id, page=3)["songs"]
    
    songs = list(filter(lambda x: len(x["featured_artists"]) == 0, songs))
    songs = list(filter(lambda x: x["artist_names"] == artists[artist_id], songs))
    
    omit_words = ["Directo", "Remix", "Mix"]
    pattern = '|'.join(map(re.escape, omit_words))
    
    songs = list(filter(lambda x: not re.search(pattern, x["full_title"], re.IGNORECASE), songs))
    
    song_ids = [song["id"] for song in songs]
    
    return song_ids

We will keep fetching songs until we have 10k words per artist.

In [118]:
texts = {}

MINIMUM_WORDS = 10000

for artist_id, artist_name in artists.items():
    wordcount = 0
    songcount = 0
    done = False
    page = 1
    texts[artist_name] = ""
    
    while not done:
        for song_id in find_songs(artist_id, page):
            song = genius.search_song(song_id=song_id)

            if song:
                text = song.to_text()
                texts[artist_name] += text

                wordcount += len(text.split())
                songcount += 1
            
                if wordcount > MINIMUM_WORDS:
                    done = True
                    break
    
        page +=1 

    print(f"Finished downloading {artist_name} words: {wordcount}, songs: {songcount} songs, {page} pages")

Finished downloading Kase.O words: 10955, songs: 8 songs, 3 pages
Finished downloading SFDK words: 10058, songs: 9 songs, 2 pages
Finished downloading Ayax y Prok words: 10100, songs: 11 songs, 3 pages
Finished downloading Kaze words: 10503, songs: 9 songs, 3 pages


Now we have one text for each artist. However, we as we can see there are some errors like `"2 ContributorsDe Una; Pt. 2: Ya No Puedo Más Lyrics"` or `"levantarEmbed"`

In [123]:
texts["Kase.O"][:1000]

'11 ContributorsComo el Sol 2011 Lyrics\n\nYeah, hell yeah\nWell, All Right!\nWell, All Right!\nBrillando hasta la E de "Horizonte" (Yeah, ahá)\n\nMe siento el progenitor de una gran prole\nEl profesor loco, el preferido del cole\nMe siento como aquel profeta multitudinario\nEl propulsor de este flow extra planetario\nPero el mensaje que te traje no lo temas\nRespeta a los demás pero a ti al que más (¡Eh!)\nA mí al que más, llevo brillando eras\nIluminando a las demás esferas\nEn sus posibles aunque pobres atmósferas mi rap es un gas\nMezclando en el oxígeno ritmos y rimas\nEstos pequeños seres viven mejor\nLlovió la inteligencia, llovió la sensibilidad\nY vieron nacer al amor (¡Uh!), y vieron reír al humor (¡Uh!)\nLlovieron los conceptos, verbos de mi verso creador\nMi estilo es como caminar por un vergel\nSe acumula, converge la belleza en este eje\nHey, en este eje\nSee Kase.O LiveGet tickets as low as $27You might also like\nMi estilo es como el sol\nNada alrededor de mi costado\nN

We are going to clean the texts

In [140]:
def clean_text(text):
    # initial processing
    text = text.lower()  # lowercase
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation

    # remove song initial message
    text = re.sub(r'contributors.+\n', '', text)
    
    # Remove special characters
    text = re.sub(r'\W', ' ', text)

    # remove Genius messages
    text = re.sub("embed", '', text)
    text = re.sub(r'see\wliveget tickets as low as', '', text)
    text = re.sub("you might also like", '', text)

    # remove extra spaces
    text = ' '.join(text.split())
    
    return text

In [169]:
for artist_name in artist_names:
    texts[artist_name] = clean_text(texts[artist_name])

In [170]:
texts["Kase.O"][:1000]

'yeah hell yeah well all right well all right brillando hasta la e de horizonte yeah ahá me siento el progenitor de una gran prole el profesor loco el preferido del cole me siento como aquel profeta multitudinario el propulsor de este flow extra planetario pero el mensaje que te traje no lo temas respeta a los demás pero a ti al que más eh a mí al que más llevo brillando eras iluminando a las demás esferas en sus posibles aunque pobres atmósferas mi rap es un gas mezclando en el oxígeno ritmos y rimas estos pequeños seres viven mejor llovió la inteligencia llovió la sensibilidad y vieron nacer al amor uh y vieron reír al humor uh llovieron los conceptos verbos de mi verso creador mi estilo es como caminar por un vergel se acumula converge la belleza en este eje hey en este eje see kaseo liveget tickets as low as mi estilo es como el sol nada alrededor de mi costado nada alrededor de mi costado mi estilo es como el sol fuera de control solo soltero y solitario como el sol nada alrededor

Now we will vectorize the words into tokens

In [185]:
nltk.download('punkt')
nltk.download('punkt_tab')

batch_size = 10

corpus = {}

for name in artist_names:
    tokens = word_tokenize(texts[name])
    corpus[name] = [tokens[i:i + batch_size] for i in range(0, len(tokens), batch_size)]

corpus["Kase.O"][:1]

[nltk_data] Downloading package punkt to /home/antonio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/antonio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


[['yeah',
  'hell',
  'yeah',
  'well',
  'all',
  'right',
  'well',
  'all',
  'right',
  'brillando']]

In [190]:
nltk.download('stopwords')
stop_words = set(stopwords.words(['english', 'spanish']))

filtered_corpus = {}

for name in artist_names:
    filtered_corpus[name] = [[word for word in doc if word not in stop_words] for doc in corpus[name]]

filtered_corpus["Kase.O"][:3]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/antonio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['yeah', 'hell', 'yeah', 'well', 'right', 'well', 'right', 'brillando'],
 ['horizonte', 'yeah', 'ahá', 'siento'],
 ['progenitor', 'gran', 'prole', 'profesor', 'loco', 'preferido']]